<a href="https://colab.research.google.com/github/Lukas4319/Animal_classification/blob/kanodap/kanodap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train & Test

In [ ]:
class Trainer:

    def train(self, epochs, save_model_path, log_wandb=False):
        for epoch in range(epochs):
            start_time = time.time()
            current_lr = self.optimizer.param_groups[0]["lr"]
            print(f"Epoch {epoch + 1}/{epochs}, LR: {current_lr}")

            train_loss, train_acc = self._run_epoch(self.train_loader, is_train=True)
            val_loss, val_acc = self._run_epoch(self.val_loader, is_train=False)

            self.history["train_loss"].append(train_loss)
            self.history["val_loss"].append(val_loss)
            self.history["train_acc"].append(train_acc)
            self.history["val_acc"].append(val_acc)

            if val_loss < self.best_loss:
                self.best_loss = val_loss
                torch.save(self.model.state_dict(), save_model_path)

            if self.scheduler:
                self.scheduler.step()

            epoch_time = time.time() - start_time
            print(
                f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
                f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%, Time: {epoch_time:.2f}s"
            )

            if log_wandb:
                wandb.log({
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "train_acc": train_acc,
                    "val_acc": val_acc,
                    "epoch": epoch,
                })

        return self.history

    def evaluate(self, test_loader):
        test_loss, test_acc = self._run_epoch(test_loader, is_train=False)
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")
        return test_loss, test_acc